In [0]:
#!pip install -q whois
import csv
import requests
from bs4 import BeautifulSoup
import math
#import whois
import os.path
from os import path
def readCsvFile():
    with open('URLs.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        readURLs = []
        kinds = []
        for row in readCSV:
            url = row[1]
            kind = row[2]
            readURLs.append(url)
            kinds.append(kind)
    csvfile.close()
    return readURLs,kinds
def createCsvFile(URLs):
    csv.register_dialect('myDialect',
    quoting=csv.QUOTE_ALL,
    skipinitialspace=True)
    with open('URLs.csv', 'w') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(["id","url","type","ip","https","spam","#.","#/","#numbers","sensitive_words","uppercase_letter","length","suspicious_character",
                         "prefix-suffix","tld","entropy"])
        for row in URLs:
            writer.writerow(row)
    f.close()
def getTLDList():
    r = requests.get("http://data.iana.org/TLD/tlds-alpha-by-domain.txt")
    soup = BeautifulSoup(r.content)
    tldList = []
    d = soup.findAll(text=True)
    for line in d:
        for line2 in line.strip().split('\n'):
            if line2:
                tldList.append(line2)
    tldList = tldList[1:]
    return tldList
def is_ip(url):
    ip = False
    length = len(url)
    for i in range(length):
        count = 0
        j = i
        while(j<length and ((ord(url[j])>=48 and ord(url[j])<=57) or ord(url[j])==46)):
            if(ord(url[j])==46):             #Ascii of the dot
                count = count + 1
            j = j + 1
        if(count==3):
            ip = True
    if(ip == True):
        return 0
    else:
        return 1
def is_https(url):
    if(url[3] == 'p' and url[4] == 's'):
        return 1
    else:
        return 0
def is_spam(url):
    u = url[:4]
    if(u == "SPAM"):
        return 0
    else:
        return 1
def numberOfDots(url):
    if(url[8]=='w'):
        if(url[4]=='s'):
            i = 12
        else:
            i = 11
    else:
        if(url[4]=='s'):
            i = 8
        else:
            i = 7
    count = 0
    length = len(url)
    while(i<length):
        if(url[i]=='.'):
            count = count + 1
        i = i + 1
    if(count>2):
        return 0
    else:
        return 1
def numberOfSlashes(url):
    length = len(url)
    count = 0
    for i in range(length):
        if(url[i] == '/'):
            count = count + 1
    if(count>6):
        return 0
    else:
        return 1
def numberOfNumbers(url):
    length = len(url)
    count = 0
    for i in range(length):
        if(ord(url[i])>=48 and ord(url[i])<=57):
            count = count + 1
    if(count>6):
        return 0
    else:
        return 1
def existenceOfSensitiveWords(url):
    phishingWords = ["secure","account","login","signin","confirm","submit"]
    phishing = False
    for i in range(len(phishingWords)):
        if(phishingWords[i] in url):
            phishing = True
    if(phishing == True):
        return 0
    else:
        return 1
def existenceOfUpperCaseLetter(url):
    length = len(url)
    phishing = False
    for i in range(length):
        if(ord(url[i])>=65 and ord(url[i])<=90):
            phishing = True
    if(phishing == True):
        return 0
    else:
        return 1
def length(url):
    if(len(url) > 100):
        return 0
    else:
        return 1
def has_suspicious_char(url):
    if(url.find("@") != -1 or url.find("&") != -1 or url.count("//")>1):
        return 0
    else:
        return 1
def prefix_suffix(url):
    if(url.find("-") != -1):
        return 0
    else:
        return 1
def TLD_count(url,tldList):
    tld_Found = False
    slash_count = 0
    if(url.count('/') == 2):
        url = url + "/"
    i = 0
    stop = 0
    while(i<len(url) and stop==0):
        if(url[i] == '/'):
            slash_count = slash_count + 1
        if(slash_count==3):
            stop = 1
        i = i + 1
    u = url[i:]
    for tld in tldList:
        if(u.find("."+tld.lower()+".") != -1 or
           u.find("."+tld.lower()+"/") != -1 or
           u.find("."+tld.lower()+"&") != -1):
            tld_Found = True
    if(tld_Found == True):
        return 0
    else: 
        return 1 
def entropy(url):
    if not url:
        return 0
    entropy = 0
    for x in range(256):
        p_x = float(url.count(chr(x)))/len(url)
        if(p_x > 0):
            entropy += - p_x*math.log(p_x, 2)
    if(entropy>4.8):
        return 0
    else:
        return 1
def domain_age(url):
    res = whois.whois(url)
    try:
        print(res.creation_date.year)
        if(res.creation_date.year>2013):
            return 0
        else:
            return 1
    except:
        print("An exception occurred")
        return 0
def alexarank(url):
    r = requests.get("https://www.alexa.com/siteinfo/"+url)
    soup = BeautifulSoup(r.content)
    links = soup.find_all("div")
    i = 0
    stop = 0
    while(i<len(links) and stop==0):
        if("This site ranks:" == links[i].text):
            rank = links[i+4].text
            rank = rank.replace(',','')
            rank = rank[4:10]
            stop = 1
        i = i + 1
    try:
        if(int(rank)<150000):
            return 1
        else:
            return 0
    except:
        return 0


url,kind = readCsvFile()
ip = []
https = []
spam = []
number_of_dots = []
number_of_slashes = []
number_of_numbers = []
sensitive_words = []
upper_case_letter = []
length_ = []
suspicious_character = []
prefixSuffix = []
tld = []
entropy_ = []
tldList = getTLDList()
for i in range(len(url)):
    ip.append(is_ip(url[i]))
    https.append(is_https(url[i]))
    spam.append(is_spam(url[i]))
    number_of_dots.append(numberOfDots(url[i]))
    number_of_slashes.append(numberOfSlashes(url[i]))
    number_of_numbers.append(numberOfNumbers(url[i]))
    sensitive_words.append(existenceOfSensitiveWords(url[i]))
    upper_case_letter.append(existenceOfUpperCaseLetter(url[i]))
    length_.append(length(url[i]))
    suspicious_character.append(has_suspicious_char(url[i]))
    prefixSuffix.append(prefix_suffix(url[i]))
    tld.append(TLD_count(url[i],tldList))
    entropy_.append(entropy(url[i]))
URLs = []
for i in range(len(url)):
    row = []
    for j in range(16):
        if (j % 16 == 0):
            row.append(i + 1)
        elif (j % 16 == 1):
            row.append(url[i])
        elif (j % 16 == 2):
            row.append(kind[i])
        elif (j % 16 == 3):
            row.append(ip[i])            
        elif (j % 16 == 4):
            row.append(https[i])
        elif (j % 16 == 5):
            row.append(spam[i])
        elif (j % 16 == 6):
            row.append(number_of_dots[i])    
        elif (j % 16 == 7):
            row.append(number_of_slashes[i])
        elif (j % 16 == 8):
            row.append(number_of_numbers[i])
        elif (j % 16 == 9):
            row.append(sensitive_words[i])
        elif (j % 16 == 10):
            row.append(upper_case_letter[i])
        elif (j % 16 == 11):
            row.append(length_[i])  
        elif (j % 16 == 12):
            row.append(suspicious_character[i])
        elif (j % 16 == 13):
            row.append(prefixSuffix[i])
        elif (j % 16 == 14):
            row.append(tld[i])
        elif (j % 16 == 15):
            row.append(entropy_[i])     
    URLs.append(row)
createCsvFile(URLs)